# iQuHack 2026 - Quantum Entanglement Distillation Game

In this game, you design quantum circuits to distill noisy Bell pairs and claim edges in a network.

**Game Flow**: Register -> Select Starting Node -> Design Circuits -> Claim Edges -> Score Points

In [ ]:
from client import GameClient
from visualization import GraphTool
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import json
from pathlib import Path

## Session Management

Save/load your session to avoid re-registering.

In [ ]:
SESSION_FILE = Path("session.json")

def save_session(client):
    if client.api_token:
        with open(SESSION_FILE, "w") as f:
            json.dump({"api_token": client.api_token, "player_id": client.player_id, "name": client.name}, f)
        print(f"Session saved.")

def load_session():
    if not SESSION_FILE.exists():
        return None
    with open(SESSION_FILE) as f:
        data = json.load(f)
    client = GameClient(api_token=data.get("api_token"))
    client.player_id = data.get("player_id")
    client.name = data.get("name")
    status = client.get_status()
    if status:
        print(f"Resumed: {client.player_id} | Score: {status.get('score', 0)} | Budget: {status.get('budget', 0)}")
        return client
    return None

In [ ]:
# Try to resume existing session
client = load_session()

if not client:
    print("No saved session. Register below.")

## Step 1: Register

Skip this if you resumed a session above.

In [ ]:
if client and client.api_token:
    print(f"Already registered as {client.player_id}")
else:
    client = GameClient()
    
    # CHANGE THESE to your unique values
    PLAYER_ID = "SleeplessShade"
    PLAYER_NAME = "Mogrel"
    
    result = client.register(PLAYER_ID, PLAYER_NAME, location=input("remote or in_person: ").strip())
    
    if result.get("ok"):
        print(f"Registered! Token: {client.api_token[:20]}...")
        candidates = result["data"].get("starting_candidates", [])
        print(f"\nStarting candidates ({len(candidates)}):")
        for c in candidates:
            print(f"  - {c['node_id']}: {c['utility_qubits']} qubits, +{c['bonus_bell_pairs']} bonus")
        save_session(client)
    else:
        print(f"Failed: {result.get('error', {}).get('message')}")

## Step 2: Select Starting Node

In [ ]:
status = client.get_status()

if status.get('starting_node'):
    print(f"Starting node: {status['starting_node']}")
    print(f"Budget: {status['budget']} | Score: {status['score']}")
else:
    print("Select a starting node from the candidates shown above.")
    # Uncomment and modify:
    result = client.select_starting_node("Singapore")
    print(result)

## Step 3: Explore the Network

In [ ]:
client.print_status()

In [ ]:
# Visualize the network (focused around your nodes)
viz = GraphTool(client.get_cached_graph())
owned = set(client.get_status().get('owned_nodes', []))
viz.render(owned)

## Step 4: Design a Distillation Circuit

For N Bell pairs, qubits are paired outside-in:
- Pair 0: qubits {0, 2N-1}
- Pair 1: qubits {1, 2N-2}
- ...
- Final pair: qubits {N-1, N}

Design a circuit that improves fidelity through LOCC operations.

In [ ]:
import numpy as np

def create_distillation_circuit(num_rounds: int = 1, use_dejmps: bool = False):
    """
    Entanglement distillation circuit supporting BBPSSW and DEJMPS variants.
    
    Args:
        num_rounds: Number of purification rounds (1-7). Uses N = num_rounds + 1 pairs.
        use_dejmps: If True, applies Rx(pi/2) twirling (DEJMPS style).
    
    Topology (N pairs, 2N qubits):
        Alice: qubits 0 to N-1
        Bob:   qubits N to 2N-1
        Pair i: (i, 2N-1-i)  [folded/nested]
        Survivor: pair N-1 = (N-1, N)
    """
    if num_rounds < 1 or num_rounds > 7:
        raise ValueError("num_rounds must be 1-7 (max 8 pairs)")
    
    num_pairs = num_rounds + 1
    total_qubits = 2 * num_pairs
    
    qr = QuantumRegister(total_qubits, 'q')
    mA = ClassicalRegister(num_rounds, 'mA')
    parity = ClassicalRegister(num_rounds, 'par')
    flag = ClassicalRegister(1, 'flag')
    qc = QuantumCircuit(qr, mA, parity, flag)
    
    def get_pair(i):
        """Returns (alice_qubit, bob_qubit) for pair i."""
        return (i, total_qubits - 1 - i)
    
    # Survivor is the innermost pair
    surv_A, surv_B = get_pair(num_rounds)
    
    # 1. Optional DEJMPS Twirling
    if use_dejmps:
        for q in range(total_qubits):
            qc.rx(np.pi/2, q)
    
    # 2. Bilateral CNOTs: Survivor controls -> Sacrificial targets
    for i in range(num_rounds):
        sac_A, sac_B = get_pair(i)
        qc.cx(surv_A, sac_A)  # Alice side
        qc.cx(surv_B, sac_B)  # Bob side
    
    # 3. Optional DEJMPS Untwirling (survivor only)
    if use_dejmps:
        qc.rx(-np.pi/2, surv_A)
        qc.rx(-np.pi/2, surv_B)
    
    # 4. Measure sacrificial pairs with feedforward
    for i in range(num_rounds):
        sac_A, sac_B = get_pair(i)
        qc.measure(sac_A, mA[i])
        with qc.if_test((mA[i], 1)):
            qc.x(sac_B)
        qc.measure(sac_B, parity[i])
    
    # 5. Compute global flag = OR(all parity bits)
    scratch = 0
    qc.reset(scratch)
    
    max_val = 1 << num_rounds
    for val in range(1, max_val):
        with qc.if_test((parity, val)):
            qc.x(scratch)
    
    qc.measure(scratch, flag[0])
    
    return qc


def get_node_value(client, node_id):
    """Calculate strategic value of a node (utility + bonus pairs)."""
    node = client.get_node_info(node_id)
    if not node:
        return 0
    return node.get('utility_qubits', 0) + 2 * node.get('bonus_bell_pairs', 0)


def get_target_node(edge, owned_nodes):
    """Get the node we'd gain by claiming this edge."""
    n1, n2 = edge['edge_id']
    return n2 if n1 in owned_nodes else n1


def score_edge(client, edge, owned_nodes):
    """Score an edge based on strategic value."""
    target = get_target_node(edge, owned_nodes)
    node_value = get_node_value(client, target)
    difficulty_penalty = edge['difficulty_rating'] * 5
    threshold_penalty = edge['base_threshold'] * 20
    return node_value - difficulty_penalty - threshold_penalty


print("✓ Circuit and scoring functions loaded!")
print(f"  2-pair circuit: {create_distillation_circuit(1).num_qubits} qubits")
print(f"  8-pair circuit: {create_distillation_circuit(7).num_qubits} qubits")

## Step 5: Claim an Edge

In [ ]:
# === ANALYZE AVAILABLE EDGES ===
def analyze_edges():
    """Show strategic analysis of available edges."""
    claimable = client.get_claimable_edges()
    owned = set(client.get_status().get('owned_nodes', []))
    
    print(f"Claimable edges: {len(claimable)}\n")
    print(f"{'Target Node':<35} {'Threshold':>10} {'Diff':>6} {'Utility':>8} {'Bonus':>6}")
    print("-" * 70)
    
    scored = []
    for edge in claimable:
        target = get_target_node(edge, owned)
        node = client.get_node_info(target)
        utility = node.get('utility_qubits', 0) if node else 0
        bonus = node.get('bonus_bell_pairs', 0) if node else 0
        edge_score = score_edge(client, edge, owned)
        scored.append((edge, target, utility, bonus, edge_score))
    
    scored.sort(key=lambda x: -x[4])
    
    for edge, target, utility, bonus, edge_score in scored[:10]:
        print(f"{target[:35]:<35} {edge['base_threshold']:>10.3f} {edge['difficulty_rating']:>6} {utility:>8} {bonus:>6}")
    
    if len(scored) > 10:
        print(f"\n... and {len(scored)-10} more edges")
    
    return scored

scored_edges = analyze_edges()

In [ ]:
# ========================================================================
#                    OPTIMIZED QUANTUM TERRITORY BOT
# ========================================================================

def optimized_bot():
    """
    Maximizes score by:
    - Prioritizing high-value nodes (utility + bonus pairs)
    - Using minimum resources needed to claim
    - Retrying failed attempts (they're FREE!)
    - Tracking F×P score for claim strength
    """
    import time
    
    MIN_BUDGET = 3
    MAX_RETRIES = 10
    
    # (num_pairs, num_rounds) ordered by cost
    CONFIGS = [
        (2, 1), (3, 2), (4, 3), (5, 4), (6, 5), (8, 7)
    ]
    
    stats = {'claimed': 0, 'spent': 0, 'attempts': 0}
    edges_exhausted = set()
    
    print("=" * 60)
    print("       OPTIMIZED QUANTUM TERRITORY BOT")
    print("=" * 60)
    
    while True:
        status = client.get_status()
        budget = status.get('budget', 0)
        score = status.get('score', 0)
        owned_nodes = set(status.get('owned_nodes', []))
        
        print(f"\n{'─'*60}")
        print(f"Budget: {budget} | Score: {score} | Nodes: {len(owned_nodes)} | Claimed: {stats['claimed']}")
        
        if budget < MIN_BUDGET:
            print("⚠ Budget exhausted!")
            break
        
        claimable = client.get_claimable_edges()
        if not claimable:
            print("🎉 No more claimable edges!")
            break
        
        available = [e for e in claimable if tuple(e['edge_id']) not in edges_exhausted]
        
        if not available:
            print("Resetting edge list...")
            edges_exhausted.clear()
            available = claimable
        
        # Smart edge selection
        available.sort(key=lambda e: (-score_edge(client, e, owned_nodes), 
                                    e['base_threshold'], 
                                    e['difficulty_rating']))
        
        target_edge = available[0]
        edge_id = tuple(target_edge['edge_id'])
        threshold = target_edge['base_threshold']
        target_node = get_target_node(target_edge, owned_nodes)
        node_info = client.get_node_info(target_node)
        
        utility = node_info.get('utility_qubits', 0) if node_info else 0
        bonus = node_info.get('bonus_bell_pairs', 0) if node_info else 0
        
        print(f"\n🎯 Target: {target_node}")
        print(f"   Threshold: {threshold:.3f} | Rewards: +{utility} utility, +{bonus} pairs")
        
        # Choose starting config based on threshold
        if threshold <= 0.70:
            start_idx = 0
        elif threshold <= 0.78:
            start_idx = 1
        elif threshold <= 0.85:
            start_idx = 2
        elif threshold <= 0.90:
            start_idx = 3
        else:
            start_idx = 4
        
        claimed = False
        best_fid = 0
        
        for num_pairs, num_rounds in CONFIGS[start_idx:]:
            if budget < num_pairs:
                continue
            
            circuit = create_distillation_circuit(num_rounds=num_rounds)
            flag_bit = 2 * num_rounds
            
            print(f"   📡 Trying {num_pairs} pairs...")
            
            for attempt in range(1, MAX_RETRIES + 1):
                stats['attempts'] += 1
                
                try:
                    result = client.claim_edge(edge_id, circuit, flag_bit, num_pairs)
                    
                    if result.get("ok"):
                        data = result["data"]
                        success = data.get("success", False)
                        fidelity = data.get("fidelity", 0)
                        prob = data.get("success_probability", 0)
                        best_fid = max(best_fid, fidelity)
                        
                        if success:
                            print(f"      ✅ CLAIMED! F={fidelity:.3f} P={prob:.3f}")
                            stats['claimed'] += 1
                            stats['spent'] += num_pairs
                            claimed = True
                            break
                        elif attempt == MAX_RETRIES:
                            print(f"      Max retries. Best F={best_fid:.3f}")
                    else:
                        print(f"      ❌ {result.get('error', {}).get('message', 'Error')}")
                        break
                        
                except Exception as e:
                    print(f"      ❌ {e}")
                    break
                
                time.sleep(0.2)
            
            if claimed:
                break
        
        if not claimed:
            print(f"   ❌ Could not claim. Skipping.")
            edges_exhausted.add(edge_id)
        
        time.sleep(0.3)
    
    # Summary
    print("\n" + "=" * 60)
    print("                    FINAL SUMMARY")
    print("=" * 60)
    final = client.get_status()
    print(f"Final Score: {final.get('score', 0)}")
    print(f"Final Budget: {final.get('budget', 0)}")
    print(f"Edges Claimed: {stats['claimed']}")
    print(f"Bell Pairs Spent: {stats['spent']}")
    if stats['spent'] > 0:
        print(f"Efficiency: {stats['claimed']/stats['spent']:.2f} edges/pair")
    print("=" * 60)

# RUN THE BOT
optimized_bot()

## Step 6: Check Progress

In [ ]:
client.print_status()

In [ ]:
# View leaderboard
leaderboard = client.get_leaderboard()["leaderboard"]
print("Leaderboard:")
for i, p in enumerate(leaderboard[:10]):
    print(f"{i+1}. {p.get('player_id', 'Unknown'):20} Score: {p.get('score', 0)}")

## Tips

- **Failed attempts are free** - only successful claims cost bell pairs
- **More bell pairs** can improve fidelity but cost more budget
- **Vertex rewards** are competitive - top players by claim strength earn rewards
- **Budget management** is key - if budget reaches 0, you're eliminated

Good luck!

In [ ]:
# Restart game (uncomment to use)
# result = client.restart()
# print(result)